<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Quickbooks - List all invoices
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Harvest/Harvest_Filtered_time_entries.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedbacks</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Harvest+-+Get+Filtered+List+of+Time+Entries:+Error+short+description">Bug report</a>

**Tags:** #quickbooks #invoice #api #list #python #get

**Author:** [Landry Christensen](https://github.com/lchristensen6)

**Last update:** 2023-09-03 (Created: 2023-09-03)

**Description:** This notebook will list all invoices from the Quickbooks API. An Invoice represents a sales form where the customer pays for a product or service later.

**References:**
- [Quickbooks API Reference](https://developer.intuit.com/app/developer/qbo/docs/api/accounting/all-entities/account)

## Input

### Import libraries

In [1]:
import requests
import pandas as pd
import naas

try:
    from intuitlib.client import AuthClient
    from intuitlib.enums import Scopes
except: 
    !pip install intuit-oauth --user
    from intuitlib.client import AuthClient
    from intuitlib.enums import Scopes

### Setup Variables

[Create your Quickbooks app and company](https://developer.intuit.com/app/developer/qbo/docs/get-started/start-developing-your-app)<br>
[Create your personnal access tokens](https://developer.intuit.com/app/developer/playground)
- `client_id`: Client ID from Quickbooks app
- `client_secret`: Client Secret from Quickbooks app
- `company_id`: Company id from the Quickbooks company you want to access (stored as `Realm ID` in the intuit playground)
- `refresh_token`: Generated refresh token to access the Quickbooks API - To generate this token follow these steps:
    - Once you've logged in to the [intuit developer playground](https://developer.intuit.com/app/developer/playground) select your app
    - Select the scope for this access token and click "Get authorization code" the page should refresh
    - Click the "Get Tokens" button and then on the right in the "response" box, copy the refresh_token  
    The steps to get a refresh token should only need to be done the first time the secret needs to be generated
- `environment`: Environment of your Quickbooks company (`sandbox` or `production`)
- `limit`: Entries limit, to get all entries enter `None`

In [2]:
client_id = naas.secret.get("QUICKBOOKS_CLIENT_ID") or "YOUR_QUICKBOOKS_CLIENT_ID"
client_secret = naas.secret.get("QUICKBOOKS_CLIENT_SECRET") or "YOUR_QUICKBOOKS_CLIENT_SECRET"
company_id = naas.secret.get("QUICKBOOKS_COMPANY_ID") or "YOUR_QUICKBOOKS_COMPANY_ID"
refresh_token = naas.secret.get("QUICKBOOKS_REFRESH_TOKEN") or "YOUR_QUICKBOOKS_REFRESH_TOKEN"
environment = "sandbox"
limit = None

In [3]:
auth_client = AuthClient(
    client_id,
    client_secret,
    'https://developer.intuit.com/v2/OAuth2Playground/RedirectUrl',
    environment, # sandbox or production
)

auth_client.refresh(refresh_token=refresh_token)

## Model

### List all invoices

This function will list all invoices from the Quickbooks API

In [4]:
# Flatten the nested dict
def flatten_dict(d, parent_key='', sep='/', num = ''):
    """
    Flattens a nested dictionary into a single level dictionary.

    Args:
        d (dict): A nested dictionary.
        parent_key (str): Optional string to prefix the keys with.
        sep (str): Optional separator to use between parent_key and child_key.

    Returns:
        dict: A flattened dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{num}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep, num='').items())
        elif isinstance(v, list):
            for i in range(len(v)):
                items.extend(flatten_dict(v[i], new_key, sep=sep, num= "_" + str(i + 1) ).items())
        else:
            items.append((new_key, v))
    return dict(items)

def list_invoices(access_token, company_id, limit=None):
    
    # Init
    data = []
    df = pd.DataFrame()
    maxresults = ''
    
    # Requests
    if limit:
        maxresults = f"maxresults {limit}"
    query = f"select * from invoice startposition 1 {maxresults}"
    url = f'https://sandbox-quickbooks.api.intuit.com/v3/company/{company_id}/query?query={query}'
    
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json",
        "Content-Type": "application/text",
    }

    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        res_json = res.json()
        queryResponse = res_json.get('QueryResponse')
        invoices = queryResponse.get("Invoice")

        for invoice in invoices:
            data.append(flatten_dict(invoice))

    df = pd.DataFrame(data)
    return df

## Output

### Display result

In [ ]:
df_invoices = list_invoices(
    auth_client.access_token, 
    company_id, limit=limit
)

print("Row fetched:", len(df_invoices))
df_invoices.head()